<p style="float:right;"><i>Created By Maroyi Bisoka on 04/02/2025</i></p>

In [335]:
import numpy as np
from copy import deepcopy
from queue import Queue

### Regression with Decision Trees: Predicting continious value

| Ear Shape(x0) | Face Shape(x1) | Whiskers(x2) | Weight(y) |
|:-------------:|:--------------:|:------------:|:----------:| 
|   Pointy   |   Round     |  Present  |    7.2    |
|   Floppy   |  Not Round  |  Present  |    8.8    |
|   Floppy   |  Round      |  Absent   |    15     |
|   Pointy   |  Not Round  |  Present  |    9.2    |
|   Pointy   |   Round     |  Present  |    8.4    |
|   Pointy   |   Round     |  Absent   |    7.6    |
|   Floppy   |  Not Round  |  Absent   |    11     |
|   Pointy   |  Round      |  Absent   |    10.2   |
|    Floppy  |   Round     |  Absent   |    18     |
|   Floppy   |  Round      |  Absent   |    20     |

In [337]:
X_train = np.array([
    [1, 1, 1],
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 1],
    [1, 1, 1],
    [1, 1, 0],
    [0, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
    [0, 1, 0]
])
y_train = np.array([ 7.2,  8.8, 15. ,  9.2,  8.4,  7.6, 11. , 10.2, 18. , 20. ])

In [338]:
def split_left_and_right(X, index_feature):
    left_side, right_side = [], []

    for i in range(len(X)):
        element = X[i]
        if element[index_feature] == 1:
            left_side.append(i)
        else:
            right_side.append(i)
            
    return left_side, right_side

In [339]:
def compute_weighted_variance(X, y, left_side, right_side):
    var_left = np.var(y[left_side], ddof=1)
    w_left = len(left_side) / len(X)

    var_right = np.var(y[right_side], ddof=1)
    w_right = len(right_side) / len(X)
    w_var = w_left*var_left + w_right*var_right
    
    return w_var

In [340]:
def compute_information_gain(X, y, left_side, right_side):
    var_root = np.var(y, ddof=1)
    w_var = compute_weighted_variance(X, y, left_side, right_side)
    
    return var_root - w_var

In [341]:
# Information gain of Ear shape (index_feature = 0)
left_side, right_side = split_left_and_right(X_train, 0)
print('Feature : Ear shape')
print(f'Left side (Pointy) :  {left_side}')
print(f'Right side (Floppy) : {right_side}')
ig = compute_information_gain(X_train, y_train, left_side, right_side)
print(f'Information Gain of Ear shape {ig:.2f}')

Feature : Ear shape
Left side (Pointy) :  [0, 3, 4, 5, 7]
Right side (Floppy) : [1, 2, 6, 8, 9]
Information Gain of Ear shape 8.84


In [342]:
class Node:
    def __init__(self):
        self.X = []
        self.var = None
        self.IG = None
        self.left = None
        self.right = None
        self.end_node = False
        self.feature_name = None
        self.feature_index = None

In [343]:
def build_tree_helper(X, y, features_names, p_index, max_depth, curr_depth):
    # If True, means we reach max depth --> create leaf node    
    if curr_depth == max_depth: 
        node = Node()
        node.end_node = True
        node.feature_name = np.mean(y)
        return node
        
    m, n = len(X), len(X[0])

    # features = ['Ear Shape', 'Face Shape', 'Whiskers']
    best_feat_idx = -1
    best_left_side, best_right_side = [], []
    best_IG = None
    
    for i in range (n):
        if i == p_index:
            continue
        left_side, right_side = split_left_and_right(X, i)
        ig = compute_information_gain(X, y, left_side, right_side)
        if best_IG is None or ig > best_IG:
            best_IG, best_feat_idx = ig, i
            best_left_side, best_right_side = left_side, right_side
    
    node = Node()
    node.feature_name = features_names[best_feat_idx]
    node.feature_index = best_feat_idx
    node.IG = best_IG
    node.X = deepcopy(X)
    
    # Build left side
    node.left = build_tree_helper(X[best_left_side], y[best_left_side], features_names, node.feature_index, max_depth, curr_depth+1)
    # Build right side
    node.right = build_tree_helper(X[best_right_side], y[best_right_side], features_names, node.feature_index, max_depth, curr_depth+1)

    return node

In [344]:
def build_tree(X, y, features_names, max_depth):
    start_depth = 0
    p_node_idx = -1
    root = build_tree_helper(X, y, features_names, p_node_idx, max_depth, start_depth)
    return root

In [345]:
# Display tree 
def level_order(root):
    q = Queue()
    print('--------------------------------------------')
    q.put(root) # endqueue
    while not q.empty():
        root = q.get() # dequeue
        if type(root.feature_name) == str:
            print(f'Node : *** {root.feature_name} ***')
        if root.left:
            print(f'if feature {root.feature_name} is 1')
            if root.left.end_node is True:
                print(f'\t Prediction : {root.left.feature_name:.2f}')
            else:
                print(f'\t Go to left: {root.left.feature_name}')
                
            q.put(root.left)
        if root.right:
            print(f'else:')
            if root.right.end_node is True:
                print(f'\t Prediction : {root.right.feature_name:.2f}')
            else:
                print(f'\t Got to right: {root.right.feature_name}')
            q.put(root.right)
        
        if type(root.feature_name) == str:
            print('--------------------------------------------')

In [346]:
max_depth = 2
features_names = ['Ear Shape', 'Face Shape', 'Whiskers', 'Weight']
root = build_tree(X_train, y_train, features_names, max_depth)

In [347]:
level_order(root)

--------------------------------------------
Node : *** Ear Shape ***
if feature Ear Shape is 1
	 Go to left: Face Shape
else:
	 Got to right: Face Shape
--------------------------------------------
Node : *** Face Shape ***
if feature Face Shape is 1
	 Prediction : 8.35
else:
	 Prediction : 9.20
--------------------------------------------
Node : *** Face Shape ***
if feature Face Shape is 1
	 Prediction : 17.67
else:
	 Prediction : 9.90
--------------------------------------------


In [348]:
def compute_prediction_helper(root, x_test):
    # If feature value is 1 : go to left else : go to right
    if root.end_node:
        return root.feature_name
    
    feature_index = root.feature_index
    direction = root.left if x_test[feature_index] == 1 else root.right

    return compute_prediction_helper(direction, x_test)

In [349]:
# Compute single prediction
def compute_prediction(root, test_example):
    prediction = compute_prediction_helper(root, test_example)
    return prediction

In [350]:
x_test = np.array([1, 1, 0])
pred_weight = compute_prediction(root, x_test)
print(f'Predicted weight for {x_test} is {pred_weight:.2f}')

Predicted weight for [1 1 0] is 8.35


In [351]:
x_test = np.array([0, 1, 0])
pred_weight = compute_prediction(root, x_test)
print(f'Predicted weight for {x_test} is {pred_weight:.2f}')

Predicted weight for [0 1 0] is 17.67
